In [ ]:
import pandas as pd
df = pd.read_csv('Anne_DNA_66.csv')
df = df[df['Class'] != 'interphase']

In [ ]:
df_x = pd.read_csv('Anne_DNA_66.csv')
trimmed_interphase = df_x[df_x['Class'] == 'interphase'].head(150)
df = pd.concat([df, trimmed_interphase],axis=0) # Add interphase but balanced

In [ ]:
y = df['Class'].values

In [ ]:
df = df.drop('TableNumber', 1)
df = df.drop('ImageNumber', 1)
df = df.drop('ObjectNumber', 1)
df = df.drop('Class', 1)
df = df.drop('Nuclei_AreaShape_EulerNumber', 1)

In [ ]:
df_norm = (df - df.mean()) / (df.max() - df.min())
df_norm.values.shape

In [ ]:
y

In [ ]:
# Convert y into Y
nb_classes = 23
import numpy as np
from keras.utils import np_utils, generic_utils

# Convert labels to numeric
y_unique = np.unique(y)
dic = {}

for i, label in enumerate(y_unique):
    dic[label] = i
print dic

y_numeric = []
for el in y:
    y_numeric += [dic[el]]
    
y_numeric # now a 2000 label vector
Y = np_utils.to_categorical(y_numeric, nb_classes)

print Y.shape

Y_train = Y

In [ ]:
X_train = df_norm.values

In [ ]:
from sklearn.cross_validation import StratifiedKFold

#print y_numeric
skl = StratifiedKFold(y_numeric, n_folds=5)

y_numeric = np.array(y_numeric)

for train,test in skl:
    print len(train), len(test)
    
X_4 = X_train[train]
Y_4 = Y[train]
y_numeric_4 = y_numeric[train]

X_eval = X_train[test]
Y_eval = Y[test]
y_numeric_eval = y_numeric[test]

X_stratified = np.append(X_4,X_eval,axis=0)
Y_stratified = np.append(Y_4,Y_eval,axis=0)

X_stratified.shape
Y_stratified.shape

In [ ]:
## Convolutional Neural Network with 2 convolutions
import keras
from keras.models import Sequential
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.layers.normalization import BatchNormalization

nb_epoch = 12

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=97))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(23))
model.add(Activation('softmax'))

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [ ]:
tf_cb = keras.callbacks.TensorBoard(log_dir='./logs/lower_learning_rate', histogram_freq=0)

# Learning rate lowering
def learning_rate(index):
    if (index > 2500):
        return 0.005
    else:
        return 0.01
l_cb = keras.callbacks.LearningRateScheduler(learning_rate)

In [ ]:
history = model.fit(X_stratified, Y_stratified, nb_epoch=2961,validation_split=0.2, show_accuracy=True, verbose=2, 
                    callbacks = [tf_cb, l_cb])

In [ ]:
import skflow
from sklearn import datasets, metrics

clf = skflow.TensorFlowDNNClassifier(hidden_units=[64,256,4096], n_classes=23, steps=5000, batch_size=len(X_4))
clf.fit(X_4, y_numeric_4, logdir='/tmp/tf_examples/model_without_dropout/')

In [ ]:
score = metrics.accuracy_score(y_numeric_eval, clf.predict(X_eval))
print("Accuracy: %f" % score)

In [ ]:
from sklearn import datasets, metrics
iris = datasets.load_iris()
iris.target.shape

In [ ]:
# Creates a graph.
import tensorflow as tf
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
clf = skflow.TensorFlowDNNClassifier(hidden_units=[64,256,8096], n_classes=23, steps=5000, batch_size=len(X_4), 
                                     learning_rate=0.05, early_stopping_rounds=1000)
clf.fit(X_4, y_numeric_4, logdir='/tmp/tf_examples/model_with_dropout_and_early_stopping/')

In [ ]:
score = metrics.accuracy_score(y_numeric_eval, clf.predict(X_eval))
print("Accuracy: %f" % score)

In [ ]:
clf.save('dropout_DNN_model')

In [ ]:
from sklearn.lda import LDA

In [ ]:
lda = LDA()

In [ ]:
lda.fit(X_4,y_numeric_4)

In [ ]:
from sklearn import datasets, metrics
score = metrics.accuracy_score(y_numeric_eval, lda.predict(X_eval))
print("Accuracy: %f" % score)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_4,y_numeric_4)

In [ ]:
from sklearn import datasets, metrics
score = metrics.accuracy_score(y_numeric_eval, rf.predict(X_eval))
print("Accuracy: %f" % score)

In [13]:
# Real data
# Load the full data
df_full = pd.read_csv('../objects.csv')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train,y_numeric)
# Get all columns of the training data
#training_values = training_df.columns.values

In [ ]:
# Get the column labels 
training_values = df.columns.values
df_full_trimmed = df_full[training_values]

In [ ]:
# Normalize
df_full_norm = (df_full_trimmed - df_full_trimmed.mean()) / (df_full_trimmed.max() - df_full_trimmed.min())
df_full_norm.values.shape

In [ ]:
df_full_norm.describe()

In [ ]:
# Too large!
# rf.predict(df_full_norm.values)

In [ ]:
matrix = df_full_norm.values

In [ ]:
matrix.shape

In [ ]:
matrix[0:,:].shape 

In [ ]:
# T

In [ ]:
indices = [5000 * i for i in range(92)]

In [ ]:
result = np.array([])
for i in range(len(indices) - 1):
    predictions = rf.predict(matrix[indices[i]:indices[i+1],:])
    result = np.append(result, predictions)

In [ ]:
result.shape

In [ ]:
np.save('rf_result.npy',result)

In [1]:
import numpy as np

In [2]:
result = np.load('rf_result.npy')

In [4]:
result

(454793,)

In [5]:
# Make it categorical to sum up the profiles
from keras.utils import np_utils
Y = np_utils.to_categorical(result, 23)

In [10]:
import pandas as pd
images = pd.read_csv('../images.csv')

In [30]:
cc = images[['Image_Metadata_Compound','Image_Metadata_Concentration']]
cc[cc['Image_Metadata_Compound'] == 'AZ-A'][cc['Image_Metadata_Concentration'] == 30.0]

/home/carmack/.local/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,Image_Metadata_Compound,Image_Metadata_Concentration
216,AZ-A,30
217,AZ-A,30
218,AZ-A,30
219,AZ-A,30
292,AZ-A,30
293,AZ-A,30
294,AZ-A,30
295,AZ-A,30
368,AZ-A,30
369,AZ-A,30


In [38]:
indexed_images = images.set_index(['TableNumber','ImageNumber'])

In [39]:
indexed_images

Cell_line  Replicate Image_Metadata_Compound  \
TableNumber ImageNumber                                                
0           1             MCF-7wt          1                    DMSO   
            2             MCF-7wt          1                    DMSO   
            3             MCF-7wt          1                    DMSO   
            4             MCF-7wt          1                    DMSO   
            41            MCF-7wt          1                    DMSO   
            42            MCF-7wt          1                    DMSO   
            43            MCF-7wt          1                    DMSO   
            44            MCF-7wt          1                    DMSO   
            53            MCF-7wt          1                   AZ138   
            54            MCF-7wt          1                   AZ138   
            55            MCF-7wt          1                   AZ138   
            56            MCF-7wt          1                   AZ138   
            57            MCF-7wt          1                   AZ138   
            58            MCF-7wt          1                   AZ138   
            59            MCF-7wt          1                   AZ138   
            60            MCF-7wt          1                   AZ138   
            61            MCF-7wt          1                   AZ138   
            62            MCF-7wt          1                   AZ138   
            63            MCF-7wt          1                   AZ138   
            64            MCF-7wt          1                   AZ138   
            65            MCF-7wt          1                   AZ138   
            66            MCF-7wt          1                   AZ138   
            67            MCF-7wt          1                   AZ138   
            68            MCF-7wt          1                   AZ138   
            69            MCF-7wt          1                   AZ138   
            70            MCF-7wt          1                   AZ138   
            71            MCF-7wt          1                   AZ138   
            72            MCF-7wt          1                   AZ138   
            81            MCF-7wt          1                    DMSO   
            82            MCF-7wt          1                    DMSO   
...                           ...        ...                     ...   
9           3319          MCF-7wt          2                    DMSO   
            3320          MCF-7wt          2                    DMSO   
            3357          MCF-7wt          2                    DMSO   
            3358          MCF-7wt          2                    DMSO   
            3359          MCF-7wt          2                    DMSO   
            3360          MCF-7wt          2                    DMSO   
            3361          MCF-7wt          3                    DMSO   
            3362          MCF-7wt          3                    DMSO   
            3363          MCF-7wt          3                    DMSO   
            3364          MCF-7wt          3                    DMSO   
            3401          MCF-7wt          3                    DMSO   
            3402          MCF-7wt          3                    DMSO   
            3403          MCF-7wt          3                    DMSO   
            3404          MCF-7wt          3                    DMSO   
            3441          MCF-7wt          3                    DMSO   
            3442          MCF-7wt          3                    DMSO   
            3443          MCF-7wt          3                    DMSO   
            3444          MCF-7wt          3                    DMSO   
            3517          MCF-7wt          3                    DMSO   
            3518          MCF-7wt          3                    DMSO   
            3519          MCF-7wt          3                    DMSO   
            3520          MCF-7wt          3                    DMSO   
            3557          MCF-7wt          3                    